### 고급 ufunc 사용

|  |  |
| -- | -- |
| reduce(x) | 연산의 연속된 적용으로 값을 집계 |
| accumulate(x) | 모든 부분적 집계값을 유지한 채 값을 집계 |
| reduceat(x, bins) | 로컬 reduce 또는 groupby. 연속된 데이터 슬라이스를 집계된 배열로 축소 |
| outer(x, y) | x와 y의 모든 원소 조합에 대해 연산 적용. 결과 배열은 x.shape + y.shape의 차원을 가짐 |

In [1]:
import numpy as np

#### ```np.add.reduce(arr)``` : 배열의 모든 원소를 더함

In [4]:
arr = np.arange(10)
np.add.reduce(arr), arr.sum()

(45, 45)

In [5]:
np.random.seed(12346)
arr = np.random.randn(5, 5)

In [6]:
arr[::2].sort(1)

In [7]:
arr[:, :-1] < arr[:, 1:]

array([[ True,  True,  True,  True],
       [False,  True, False, False],
       [ True,  True,  True,  True],
       [ True, False,  True,  True],
       [ True,  True,  True,  True]])

#### ```np.logical_and.reduce()``` : 배열의 각 행에 있는 값이 정렬된 상태인지 검사. ```.all()```메서드와 동일

In [ ]:
np.logical_and.reduce(arr[:, :-1] < arr[:, 1:], axis = 1)

#### ```np.add.accumulate()``` : 누계를 담고 있는 같은 크기의 배열을 생성

In [8]:
arr = np.arange(15).reshape((3, 5))
np.add.accumulate(arr, axis = 1)

array([[ 0,  1,  3,  6, 10],
       [ 5, 11, 18, 26, 35],
       [10, 21, 33, 46, 60]])

#### ```np.multiply.outer()``` : 두 배열 간의 벡터곱 (외적)을 계산

In [10]:
arr = np.arange(3).repeat([1, 2, 2])
arr, np.multiply.outer(arr, np.arange(5))

(array([0, 1, 1, 2, 2]),
 array([[0, 0, 0, 0, 0],
        [0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4],
        [0, 2, 4, 6, 8],
        [0, 2, 4, 6, 8]]))

In [11]:
x, y = np.random.randn(3, 4), np.random.randn(5)
result = np.subtract.outer(x, y)
result.shape

(3, 4, 5)

#### ```np.add.reduceat()``` : groupby 연산으로 배열의 슬라잉스를 모두 집계

In [12]:
arr = np.arange(10)
np.add.reduceat(arr, [0, 5, 8])

array([10, 18, 17])

In [16]:
arr = np.multiply.outer(np.arange(4), np.arange(5))
arr, np.add.reduceat(arr, [0, 2, 4], axis = 1)

(array([[ 0,  0,  0,  0,  0],
        [ 0,  1,  2,  3,  4],
        [ 0,  2,  4,  6,  8],
        [ 0,  3,  6,  9, 12]]),
 array([[ 0,  0,  0],
        [ 1,  5,  4],
        [ 2, 10,  8],
        [ 3, 15, 12]]))

### 
### 사용자 정의 ufunc 작성

### ```np.frompyfunc()``` : 입력과 출력에 대한 표준과 함께 파이썬 함수를 인자로 취함

- 원소별로 합을 구하는 함수

In [19]:
def add_elements(x, y):
    return x + y

add_them = np.frompyfunc(add_elements, 2, 1)
add_them(np.arange(8), np.arange(8))

array([0, 2, 4, 6, 8, 10, 12, 14], dtype=object)

### ```np.vectorize()``` : 반환할 결과의 자료형을 지정 가능

In [20]:
add_them = np.vectorize(add_elements, otypes = [np.float64])
add_them(np.arange(8), np.arange(8))

array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14.])

### 
### 구조화된 배열과 레코드 배열
- 구조화된 배열 : 배열의 각 원소가 C의 구조체 혹은 다양한 이름의 필드를 갖는 SQL 테이블의 한 행으로 표현되는 것으로 생각할 수 있는 ndarray

In [23]:
dtype = [('x', np.float64), 
         ('y', np.int32)]
sarr = np.array([(1.5, 6), (np.pi, -2)], dtype = dtype)
sarr

array([(1.5       ,  6), (3.14159265, -2)],
      dtype=[('x', '<f8'), ('y', '<i4')])

In [28]:
print(sarr[0])
print(sarr[0]['y'])

(1.5, 6)
6


In [25]:
sarr['x']

array([1.5       , 3.14159265])

### 
### 중첩된 dtype과 다차원 필드

In [29]:
dtype = [('x', np.int64, 3), 
         ('y', np.int32)]
arr = np.zeros(4, dtype = dtype)
arr

array([([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0), ([0, 0, 0], 0)],
      dtype=[('x', '<i8', (3,)), ('y', '<i4')])

In [30]:
arr[0]['x']

array([0, 0, 0], dtype=int64)

In [31]:
arr['x']

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]], dtype=int64)

- 복잡한 중첩 구조를 하나의 배열 안에서 단일 메모리로 표현
- **중첩된 dtype**

In [32]:
dtype = [('x', [('a', 'f8'), ('b', 'f4')]), 
         ('y', np.int32)]
data = np.array([((1, 2), 5), ((3, 4), 6)], dtype = dtype)
data['x']

array([(1., 2.), (3., 4.)], dtype=[('a', '<f8'), ('b', '<f4')])

In [34]:
data['y']

array([5, 6])

In [35]:
data['x']['a']

array([1., 3.])

### 
### 정렬

### ```.sort()```

In [36]:
arr = np.random.randn(6)
arr.sort()
arr

array([-1.08199644,  0.37588273,  0.80139193,  1.13969136,  1.28881614,
        1.84126094])

- 첫 번째 열의 값을 정렬

In [37]:
arr = np.random.randn(3, 5)
# arr[:, 0].sort() 
arr

array([[-0.33176812, -1.47108206,  0.87050269, -0.08468875, -1.13286962],
       [-1.01114869, -0.34357617,  2.17140268,  0.12337075, -0.01893118],
       [ 0.17731791,  0.7423957 ,  0.85475634,  1.03797268, -0.32899594]])

### ```np.sort()``` : 정렬된 배열의 복사본 생성

In [46]:
arr = np.random.randn(5)
arr, np.sort(arr)

(array([-0.36360302, -0.13775933,  2.17773731, -0.47280687,  0.8356152 ]),
 array([-0.47280687, -0.36360302, -0.13775933,  0.8356152 ,  2.17773731]))

In [50]:
arr = np.random.randn(3, 5)
arr

array([[ 0.42201554,  0.11865768,  1.13518793,  1.43627776, -1.24869674],
       [ 0.19093913, -1.0984421 ,  0.78863313, -0.58272508,  1.15921734],
       [-1.71926946,  0.38535846, -2.01525772, -1.60990436, -1.20974317]])

In [52]:
arr.sort(axis = 1)
arr

array([[-1.24869674,  0.11865768,  0.42201554,  1.13518793,  1.43627776],
       [-1.0984421 , -0.58272508,  0.19093913,  0.78863313,  1.15921734],
       [-2.01525772, -1.71926946, -1.60990436, -1.20974317,  0.38535846]])

In [43]:
arr[:, ::-1]

array([[ 1.33885804,  0.91108374,  0.59545348, -0.18715572, -0.26822958],
       [ 1.19251887,  1.00543901, -0.19893404, -0.32150045, -0.51683937],
       [ 0.60709023,  0.39691349, -0.21707838, -0.22215536, -1.76381537]])

### 
### 간접 정렬 : argsort, lexsort
- 하나 이상의 키를 기준으로 데이터를 정렬

### ```.argsort()``` : 주어진 단일 키 혹은 여러 키 (배열이나 여러 개의 값)로 데이터 정렬시, 어떤 순서로 나열해야 하는지 알려주는 정수 인덱스 배열 반환
### ```.lexsort()``` : argsort와 유사하지만 다중 키 배열에 대해 간점 사전순 정렬 수행

In [56]:
values = np.array([5, 0, 1, 3, 2])
indexer = values.argsort()
indexer, values[indexer]

(array([1, 2, 4, 3, 0], dtype=int64), array([0, 1, 2, 3, 5]))

In [57]:
arr = np.random.randn(3, 5)
arr[0] = values
arr, arr[:, arr[0].argsort()]

(array([[ 5.        ,  0.        ,  1.        ,  3.        ,  2.        ],
        [ 0.78439331,  0.99262902, -0.48061085,  0.52089072, -0.8444945 ],
        [-1.40635117,  1.5642717 ,  0.17802672, -1.33879053,  1.3941583 ]]),
 array([[ 0.        ,  1.        ,  2.        ,  3.        ,  5.        ],
        [ 0.99262902, -0.48061085, -0.8444945 ,  0.52089072,  0.78439331],
        [ 1.5642717 ,  0.17802672,  1.3941583 , -1.33879053, -1.40635117]]))

In [58]:
first_name = np.array(['Bob', 'Jane', 'Steve', 'Bill', 'Barbara'])
last_name = np.array(['Jones', 'Arnold', 'Arnold', 'Jones', 'Walters'])
sorter = np.lexsort((first_name, last_name))
sorter

array([1, 2, 3, 0, 4], dtype=int64)

In [59]:
zip(last_name[sorter], first_name[sorter])

### 
### 대안 정렬 알고리즘
### 견고한 (stable) 정렬 알고리즘은 동일한 원소의 상대적인 위치를 그대로 둠
- 상대적인 순서가 의미를 가지는 간접 정렬의 경우 중요한 기능

| 종류 | 속도 | 견고함 | 공간복잡도 | 시간복잡도 |
| -- | -- | -- | --| -- |
| 'quicksort' | 1 | No | 0 | 0 (n^2) |
| 'mergesort' | 2 | Yes | n / 2 | 0 (n log n) |
| 'heapsort' | 3 | No | 0 | 0 (n log n) |

In [60]:
values = np.array(['2:first', '2:second', '1:first', '1:second', '1:third'])
key = np.array([2, 2, 1, 1, 1])
indexer = key.argsort(kind = 'mergesort')
indexer, values.take(indexer)

(array([2, 3, 4, 0, 1], dtype=int64),
 array(['1:first', '1:second', '1:third', '2:first', '2:second'],
       dtype='<U8'))

### 
### 배열 일부만 정렬

### ```np.partition(arr, n)``` : 반환된 결과 배열의 첫 n개의 원소는 해당 배열에서 가장 작은 값 (n개만 정렬)
### ```np.argpartition(arr, n)``` : np.argsort()와 유사. 해당 원소의 위치를 반환

In [61]:
np.random.seed(12345)
arr = np.random.randn(20)
arr, np.partition(arr, 3)

(array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057,
         1.39340583,  0.09290788,  0.28174615,  0.76902257,  1.24643474,
         1.00718936, -1.29622111,  0.27499163,  0.22891288,  1.35291684,
         0.88642934, -2.00163731, -0.37184254,  1.66902531, -0.43856974]),
 array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
        -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
         1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
         0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474]))

In [62]:
indices = np.argpartition(arr, 3)
indices, arr.take(indices)

(array([16, 11,  3,  2, 17, 19,  0,  7,  8,  1, 10,  6, 12, 13, 14, 15,  5,
         4, 18,  9], dtype=int64),
 array([-2.00163731, -1.29622111, -0.5557303 , -0.51943872, -0.37184254,
        -0.43856974, -0.20470766,  0.28174615,  0.76902257,  0.47894334,
         1.00718936,  0.09290788,  0.27499163,  0.22891288,  1.35291684,
         0.88642934,  1.39340583,  1.96578057,  1.66902531,  1.24643474]))

### ```np.searchsorted()``` : 정렬된 배열에 새로운 값을 삽입할 때, 정렬된 상태를 계속 유지하기 위한 위치를 반환

In [63]:
arr = np.array([0, 1, 7, 12, 15])
arr.searchsorted(9)

3

In [64]:
arr.searchsorted([0, 8, 11, 16])

array([0, 3, 3, 5], dtype=int64)

In [65]:
arr = np.array([0, 0, 0, 1, 1, 1, 1])
arr.searchsorted([0, 1])

array([0, 3], dtype=int64)

In [66]:
arr.searchsorted([0, 1], side = 'right')

array([3, 7], dtype=int64)

#### 
- 0부터 10000까지의 값을 특정 구간별로 나눈 배열

In [67]:
data = np.floor(np.random.uniform(0, 10000, size = 50))
bins = np.array([0, 100, 1000, 5000, 10000])
data

array([9940., 6768., 7908., 1709.,  268., 8003., 9037.,  246., 4917.,
       5262., 5963.,  519., 8950., 7282., 8183., 5002., 8101.,  959.,
       2189., 2587., 4681., 4593., 7095., 1780., 5314., 1677., 7688.,
       9281., 6094., 1501., 4896., 3773., 8486., 9110., 3838., 3154.,
       5683., 1878., 1258., 6875., 7996., 5735., 9732., 6340., 8884.,
       4954., 3516., 7142., 5039., 2256.])

- 각 데이터가 어떤 구간에 속해야 하는지 탐색하기 위해 searchsorted 메서드 사용

In [68]:
labels = bins.searchsorted(data)
labels

array([4, 4, 4, 3, 2, 4, 4, 2, 3, 4, 4, 2, 4, 4, 4, 4, 4, 2, 3, 3, 3, 3,
       4, 3, 4, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 4, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 4, 4, 3], dtype=int64)

In [70]:
import pandas as pd
pd.Series(data).groupby(labels).mean()

2     498.000000
3    3064.277778
4    7389.035714
dtype: float64